In [24]:
#!pip install instaloader

In [5]:
import pandas as pd
import instaloader
from datetime import datetime
import time
import os

In [6]:
df = pd.read_csv('data.csv')
df

,bioguide_id,state,party,full_name,url,twitter,twitter_id,facebook,id.bioguide,social.twitter,social.facebook,social.youtube_id,social.twitter_id,social.youtube,social.instagram,social.instagram_id
0,C000127,WA,Democrat,Maria Cantwell,https://www.cantwell.senate.gov,SenatorCantwell,117501995.0,senatorcantwell,C000127,SenatorCantwell,senatorcantwell,UCN52UDqKgvHRk39ncySrIMw,117501995.0,SenatorCantwell,NaN,NaN
1,K000367,MN,Democrat,Amy Klobuchar,https://www.klobuchar.senate.gov,SenAmyKlobuchar,22044727.0,NaN,K000367,SenAmyKlobuchar,NaN,UCvdeJsDsV51tFb_hVqvtYGA,22044727.0,senatorklobuchar,NaN,NaN
2,S000033,VT,Independent,Bernard Sanders,https://www.sanders.senate.gov,SenSanders,29442313.0,senatorsanders,S000033,SenSanders,senatorsanders,UCD_DaKNac0Ta-2PeHuoQ1uA,29442313.0,senatorsanders,NaN,NaN
3,W000802,RI,Democrat,Sheldon Whitehouse,https://www.whitehouse.senate.gov,SenWhitehouse,242555999.0,SenatorWhitehouse,W000802,SenWhitehouse,SenatorWhitehouse,UCnG0N70SNBkNqvIMLodPTIA,242555999.0,SenatorWhitehouse,SenWhitehouse,NaN
4,B001261,WY,Republican,John Barrasso,https://www.barrasso.senate.gov,SenJohnBarrasso,202206694.0,johnbarrasso,B001261,SenJohnBarrasso,johnbarrasso,UC80PfiCAB2Fe1MFayWlR6GQ,202206694.0,barrassowyo,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,P000622,FL,Republican,Jimmy Patronis,https://patronis.house.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,F000484,FL,Republican,Randy Fine,https://fine.house.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537,W000831,VA,Democrat,James R. Walkinshaw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,G000606,AZ,Democrat,Adelita S. Grijalva,https://grijalva.house.gov/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
insta_accounts = df[df['social.instagram'].notna()]['social.instagram'].to_list()
len(insta_accounts)

412

In [ ]:
def load_cache(cache_file='instagram_cache.csv'):
    if os.path.exists(cache_file):
        try:
            cache_df = pd.read_csv(cache_file)
            print(f"✓ Loaded {len(cache_df)} cached profiles from {cache_file}")
            return cache_df
        except Exception as e:
            print(f"Warning: Could not load cache file: {e}")
            return pd.DataFrame()
    else:
        print(f"No cache file found. Will create {cache_file}")
        return pd.DataFrame()

def save_cache(data, cache_file='instagram_cache.csv'):
    """Save Instagram data to CSV cache file."""
    try:
        df = pd.DataFrame(data)
        df.to_csv(cache_file, index=False)
        print(f"✓ Saved {len(df)} profiles to {cache_file}")
    except Exception as e:
        print(f"Warning: Could not save cache: {e}")

def get_follower_count(username, loader):
    try:
        profile = instaloader.Profile.from_username(loader.context, username)
        
        result = {
            'username': profile.username,
            'full_name': profile.full_name,
            'followers': profile.followers,
            'following': profile.followees,
            'posts': profile.mediacount,
            'is_private': profile.is_private,
            'is_verified': profile.is_verified,
            'biography': profile.biography,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        return result
        
    except instaloader.exceptions.ProfileNotExistsException:
        print(f"Error: Profile '{username}' does not exist.")
        return None
    except instaloader.exceptions.ConnectionException as e:
        print(f"Error: Connection issue - {e}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

def get_multiple_followers(usernames, delay=5, cache_file='instagram_cache.csv', use_cache=True):
    cache_df = load_cache(cache_file) if use_cache else pd.DataFrame()
    
    L = instaloader.Instaloader()
    
    results = []
    cached_usernames = set(cache_df['username'].tolist()) if not cache_df.empty and 'username' in cache_df.columns else set()
    
    if use_cache and not cache_df.empty:
        for username in usernames:
            if username in cached_usernames:
                cached_row = cache_df[cache_df['username'] == username].iloc[0].to_dict()
                results.append(cached_row)
                print(f"[CACHED] {username}: {int(cached_row['followers']):,} followers")
    
    usernames_to_fetch = [u for u in usernames if u not in cached_usernames]
    
        
    for i, username in enumerate(usernames_to_fetch):
        
        data = get_follower_count(username, L)
        
        if data:
            results.append(data)
            print(f"✓ {data['username']}: {data['followers']:,} followers")
            
            save_cache(results, cache_file)
        else:
            print(f"✗ Failed to fetch data for {username}")
        
        if i < len(usernames_to_fetch) - 1:  # Don't delay after the last request
            print(f"Waiting {delay} seconds before next request...")
            time.sleep(delay)
    
    return results

In [ ]:
all_data = get_multiple_followers(insta_accounts, delay=8, cache_file='instagram_cache.csv', use_cache=True)

=== Single Account Example ===

Profile: @instagram
Full Name: Instagram
Followers: 697,830,113
Following: 305
Posts: 8,263
Verified: Yes
Private: No
Bio: Discover what's new on Instagram 🔎✨


=== Multiple Accounts Example ===

Fetching data for: SenWhitehouse
✓ senwhitehouse: 267,340 followers

Fetching data for: senatorwicker
✓ senatorwicker: 15,440 followers

Fetching data for: sensusancollins
✓ sensusancollins: 23,942 followers

=== Summary ===
senwhitehouse: 267,340 followers
senatorwicker: 15,440 followers
sensusancollins: 23,942 followers
